In [1]:
# リスト 2.2.5  Janomeカスタム辞書利用前
# ------------------------------------------------------------------------------------------------
# 目的:
#   ・純Python製の形態素解析器 Janome を、デフォルト（同梱）辞書で動かし、
#     解析結果（表層形＋品詞などの素性列）を可視化するベースラインを確認する。
#
# 背景・理論メモ（挙動は変えず説明のみ）:
#   ・Janome は IPADIC 系フォーマット相当の辞書と遷移コストに基づき、Viterbi で最小コスト経路の
#     形態素列を求める。したがって「どこで切るか／何の品詞か」は辞書とその版に依存する（再現性のため
#     バージョンを固定・記録するのが望ましい）。
#   ・本コードは“カスタム辞書適用前（= デフォルト辞書）”の基準出力を得るためのもの。
#     後でユーザー辞書を適用した結果と比較し、固有名詞（例: 「はくたか」）や複合表現（例: 「令和元年6月1日」）
#     の一語化／分割の差を評価する際の対照として使う。
#   ・Tokenizer(wakati=False) の tokenize は Token オブジェクト列を返し、print(token) は
#     「表層形<TAB>feature,feature,...」という文字列表現（IPADIC 互換の列順）を出力する。
#     機械処理では token.surface / token.base_form / token.part_of_speech / token.reading 等の
#     プロパティを直接参照するのが堅牢（ここでは可視確認のため文字列表現をそのまま出力）。
#   ・記号（「。」など）も1トークンとして出現する。記号を残す/除くの方針は下流処理（学習・検索など）に影響するため、
#     プロジェクトとして仕様化しておくとよい。
#   ・未知語（OOV）が多いドメインでは、後段でユーザー辞書の導入により分割の安定性を高められる。
# ------------------------------------------------------------------------------------------------

# Janome サンプルコード
from janome.tokenizer import Tokenizer

# Tokenizer の生成（デフォルト辞書を使用）
# - カスタム辞書適用前の“基準”を得るため、ここでは追加オプション無しで初期化する。
# - 後でユーザー辞書を使う場合は UserDictionary を用意し、Tokenizer(userdic=...) などで指定する。
t1 = Tokenizer()

# 解析対象文（年号＋日付＋列車名を含み、辞書カバレッジの差が出やすい例）
text = "令和元年6月1日に特急はくたかに乗ります。"

# 解析実行
# - tokenize は Token オブジェクト列を返す。
# - print(token) は「表層形<TAB>品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音」の文字列表現。
for token in t1.tokenize(text):
    print(token)

# 参考（構造化アクセスの例。可視確認用なので実行はしない）:
# for token in t1.tokenize(text):
#     surface = token.surface           # 表層形
#     base    = token.base_form         # 原形
#     pos     = token.part_of_speech    # 品詞（カンマ区切りの大分類～細分類）
#     yomi    = token.reading           # 読み
#     # print(surface, base, pos, yomi)

令和	名詞,固有名詞,一般,*,*,*,令和,レイワ,レイワ
元年	名詞,一般,*,*,*,*,元年,ガンネン,ガンネン
6	名詞,数,*,*,*,*,6,*,*
月	名詞,一般,*,*,*,*,月,ツキ,ツキ
1	名詞,数,*,*,*,*,1,*,*
日	名詞,接尾,助数詞,*,*,*,日,ニチ,ニチ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
特急	名詞,一般,*,*,*,*,特急,トッキュウ,トッキュー
はく	動詞,自立,*,*,五段・カ行イ音便,基本形,はく,ハク,ハク
たか	名詞,非自立,一般,*,*,*,たか,タカ,タカ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
乗り	動詞,自立,*,*,五段・ラ行,連用形,乗る,ノリ,ノリ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
。	記号,句点,*,*,*,*,。,。,。


In [2]:
# リスト 2.2.7  Jamomeカスタム辞書利用後
# ------------------------------------------------------------------------------------------------
# 目的:
#   ・（正しくは “Janome”）のユーザー辞書を読み込んだ後の形態素解析結果を可視化し、
#     デフォルト辞書時（リスト 2.2.5）との分割境界・品詞付与の差を確認するベースを作る。
#
# 理論メモ（挙動は変えず説明のみ）:
#   ・Janome は IPADIC 互換フォーマット相当の辞書＋連接コストに基づき Viterbi で最小経路を選ぶ。
#     → どこで切るか／何の品詞かは「辞書とその版」に強く依存する（= 再現性のために版を固定・記録する）。
#   ・ユーザー辞書は「既定辞書に前置して検索させる“追加エントリ集合”」のイメージ。
#     → ドメイン語（新語・固有名詞）を一語化して OOV（未知語）率を下げ、分割の決定性を上げる役割。
#   ・Tokenizer の第1引数に文字列を渡すと **“ユーザー辞書のパス”** として扱われる（Janomeの仕様）。
#     何も指定しない場合は同梱の既定辞書のみで解析される。
#   ・ユーザー辞書 CSV の“型”には2系統がある:
#       - ipadic（デフォルト）: IPADIC 互換の多列CSV（left_id/right_id/cost/品詞…読/発 など）。
#       - simpledic            : 簡易CSV（列が少ない）。**この形式を使う時は `udic_type='simpledic'` を必ず指定**。
#     ※ 本コードは `Tokenizer('userdict.csv')` としか指定していないため、**CSV は ipadic 互換形式であることが前提**。
#       simpledic で作っているなら、下の参考行のように `udic_type='simpledic'` を明示すること。
#   ・文字コードは既定で 'utf8'。BOM なし、最終行に改行ありが安全。
#
# 実務ノート:
#   ・ユーザー辞書の効果検証は「前/後の差分」を定量化して判断する（編集距離、Jaccard、OOV率、一語化率など）。
#   ・再現性向上のため、実行時に Janome のバージョンとユーザー辞書パス（ハッシュ）をログへ残すと良い。
#   ・“令和元年6月1日”“はくたか”のような複合表現・固有名詞は、エントリ追加で一語化が期待できる。
#     例: （ipadic 互換の概形。実際の列数・順序はIPADIC準拠で要作成）
#       はくたか,*,*,*,名詞,固有名詞,一般,*,*,*,はくたか,ハクタカ,ハクタカ
#       令和元年6月1日,*,*,*,名詞,固有名詞,一般,*,*,*,令和元年6月1日,レイワガンネンロクガツツイタチ,レイワガンネンロクガツツイタチ
#
# 参考（simpledic を使いたい場合の初期化例・ここでは実行しない）:
#   from janome.tokenizer import Tokenizer
#   t2 = Tokenizer('userdict.csv', udic_type='simpledic', udic_enc='utf8')
#   # simpledic のフォーマットは Janome のドキュメントに従う（表層,品詞,読み などの少列版）
# ------------------------------------------------------------------------------------------------

# Janome サンプルコード
from janome.tokenizer import Tokenizer

t2 = Tokenizer(
    "userdict.csv"
)  # ← ipadic互換CSVとして解釈される前提（simpledicなら udic_type を明示する）

# 解析対象文（年号＋日付＋列車名を含み、ユーザー辞書での一語化効果が出やすい例）
text = "令和元年6月1日に特急はくたかに乗ります。"

# 解析実行
# - tokenize は Token オブジェクト列を返す。print(token) は「表層\tfeature...」の人間可読表現を出力。
# - 差分確認のコツ: リスト 2.2.5 の出力を保存しておき、トークン列（表層 or 原形）の LCS/Jaccard を比較する。
for token in t2.tokenize(text):
    print(token)

# 参考（構造化アクセスの例。可視化だけなら不要だが、下流処理では推奨）
# for token in t2.tokenize(text):
#     surface = token.surface
#     base    = token.base_form
#     pos     = token.part_of_speech
#     yomi    = token.reading
#     # TODO: DataFrame 化し、前/後で groupby して一語化率やOOV率の差分を集計する

令和	名詞,固有名詞,一般,*,*,*,令和,レイワ,レイワ
元年	名詞,一般,*,*,*,*,元年,ガンネン,ガンネン
6	名詞,数,*,*,*,*,6,*,*
月	名詞,一般,*,*,*,*,月,ツキ,ツキ
1	名詞,数,*,*,*,*,1,*,*
日	名詞,接尾,助数詞,*,*,*,日,ニチ,ニチ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
特急	名詞,一般,*,*,*,*,特急,トッキュウ,トッキュー
はくたか	名詞,固有名詞,*,*,*,*,はくたか,ハクタカ,ハクタカ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
乗り	動詞,自立,*,*,五段・ラ行,連用形,乗る,ノリ,ノリ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
。	記号,句点,*,*,*,*,。,。,。
